# Explorative Datenanalyse

- Erkunden Sie den bereitgestellten Datensatz mithilfe beschreibender Statistiken (z. B. Mittelwerte, Standardabweichungen, Min-/Max-Werte, fehlende Werte) und Visualisierungen (z. B. Histogramme, Boxplots)
- Präsentieren Sie die Erkenntnisse, die Sie durch die Erkundung gewonnen haben
- Weisen Sie auch darauf hin, welche Datenqualitätsprobleme Sie in Bezug auf Folgendes identifiziert haben:
- Fehlende Werte
- Ausreißer
- Zu transformierende Merkmale (z. B. Normalisierungstransformation)
- Zu entfernende Merkmale (Merkmalsauswahl)
- Andere Erkenntnisse, die in den folgenden Phasen Aufmerksamkeit erfordern
- Dokumentieren und präsentieren Sie Ihre Erkenntnisse aus der Codierung für diese Aufgabe

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno

#from sklearn import linear_model
#from sklearn.model_selection import train_test_split
#import pickle

#from sklearn.impute import SimpleImputer
#from sklearn.impute import KNNImputer

## Überblick

In [ ]:
df_step0 = pd.read_csv('../../data/raw/triathlon_watch_training_data_final.csv')
# Rename columns for better handling
df_step0.columns = df_step0.columns.str.strip().str.lower().str.replace(' ', '_')
df_step0.info

In [ ]:
# Check for missing values
msno.matrix(df_step0)

In [18]:
#liste aller Spalten, die als Object definiert sind und möglicherweise als Kategorial einzustufen sind
category_columns = list(df_step0.describe(include='object').columns)
category_columns.remove('id')
category_columns.remove('most_current_software_update')
category_columns.remove('synchronisation')
date_columns = ['most_current_software_update']
bool_columns = ['synchronisation']
target_columns = ['user_of_latest_model']
#Numerische Spalten identifizieren und Zielspalten entfernen
numeric_columns = df_step0.describe().columns
numeric_columns = [x for x in numeric_columns if x not in target_columns]
text_columns = ['id']
#Übersicht aller Spaltengruppen ausgeben
print("Category Columns:", category_columns)
print("Date Columns:", date_columns)
print("Boolean Columns:", bool_columns)
print("Numeric Columns:", numeric_columns)
print("Text Columns:", text_columns)
print("Target Columns:", target_columns)

#Boolesche Spalten umwandeln in numerische Variable
for bool_column in bool_columns:
    df_step0[bool_column] = df_step0[bool_column].map({'Yes': 1, 'No': 0})

Category Columns: ['sex', 'ctry', 'town', 'goal_of_training', 'preferred_training_daytime', 'subscription_type', 'color_of_watch']
Date Columns: ['most_current_software_update']
Boolean Columns: ['synchronisation']
Numeric Columns: ['age_of_customer', 'swimming_hours_per_week', 'biking_hours_per_week', 'running_hours_per_week', 'total_training_hours_per_week', 'vo2_max', '10k_running_time_prediction', 'calories_burned_per_week', 'support_cases_of_customer', 'customer_years']
Text Columns: ['id']
Target Columns: ['user_of_latest_model']


# Step 0

In [ ]:
#import pandas as pd
#import numpy as np

# Datei einlesen
#file_path = "triathlon_watch_training_data_final.csv"  # Passe den Pfad an
#df_step0 = pd.read_csv(file_path)

# Zielvariable definieren
target_column = "user_of_latest_model" if "user_of_latest_model" in df_step0.columns else df_step0.columns[-1]

# Zeilen mit fehlenden Werten in der Zielvariable entfernen
df_step0 = df_step0.dropna(subset=[target_column])

# Zeilen mit mehr als zwei fehlenden Werten entfernen
df_step0 = df_step0[df_step0.isnull().sum(axis=1) <= 2]

# Fehlende Werte analysieren
missing_values = df_step0.isnull().sum()
missing_values = missing_values[missing_values > 0]


# Fehlende Werte durch Durchschnitt (numerisch) oder Modus (kategorisch) ersetzen
#df_step0 = df_step0.copy()

numeric_cols = df_step0.select_dtypes(include=[np.number]).columns
categorical_cols = df_step0.select_dtypes(exclude=[np.number]).columns

for col in numeric_cols:
    df_step0[col] = df_step0[col].fillna(df_step0[col].mean())

for col in categorical_cols:
    df_step0[col] = df_step0[col].fillna(df_step0[col].mode()[0])

# Speichern der bereinigten Daten
output_path = "triathlon_watch_training_data_step0.csv"
df_step0.to_csv(output_path, index=False)

# Ergebnis anzeigen
print("Step 0 abgeschlossen. Bereinigte Daten gespeichert unter:", output_path)
print(df_step0.head())



In [ ]:
msno.matrix(df_step0)

In [25]:
numerical_columns = df_step0.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df_step0.select_dtypes(exclude=['number']).columns.tolist()

print("📊 Numerische Spalten:", numerical_columns)
print("🔠 Kategoriale Spalten:", categorical_columns)


📊 Numerische Spalten: ['age_of_customer', 'swimming_hours_per_week', 'biking_hours_per_week', 'running_hours_per_week', 'total_training_hours_per_week', 'vo2_max', '10k_running_time_prediction', 'calories_burned_per_week', 'support_cases_of_customer', 'customer_years', 'synchronisation', 'user_of_latest_model']
🔠 Kategoriale Spalten: ['id', 'sex', 'ctry', 'town', 'most_current_software_update', 'goal_of_training', 'preferred_training_daytime', 'subscription_type', 'color_of_watch']


In [8]:
# One-Hot-Encoding für alle kategorialen Variablen
df_step0_encoded = pd.get_dummies(df_step0, columns=categorical_cols, drop_first=True, dtype=float)
